In [27]:
#%pip install -U fastmcp

<div class="alert alert-block alert-info" style="background-color: #05299E; color: white;">
    <h1 style="color: white;">Semantic Kernel Client</h1>
    <h4 style="color: white;">Cross-framework test for Semantic Kernel <<\MCP>> OpenAI Agents</h4>
    <h6 style="color: white;">@author: Cher Han</h6>
</div>

In [28]:
#from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.connectors.mcp import MCPSsePlugin


from semantic_kernel import Kernel

from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread


from semantic_kernel.contents import ChatMessageContent, FunctionCallContent, FunctionResultContent
from semantic_kernel.functions import kernel_function

In [29]:
import asyncio
import json 
import nest_asyncio
import operator
import os
import sys
from dataclasses import dataclass, field

from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from pydantic import BaseModel, Field
from typing import Annotated, Any, Dict, List, Tuple, TypedDict, Optional
from dotenv import load_dotenv




In [30]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
AZURE_MCP_HHAI_ENDPOINT = os.getenv("AZURE_MCP_HHAI_ENDPOINT")
AZURE_MCP_HHAI_API_KEY = os.getenv("AZURE_MCP_HHAI_API_KEY")

In [31]:
nest_asyncio.apply()

In [32]:
kernel = Kernel()

In [33]:
kernel.add_service(OpenAIChatCompletion(ai_model_id="gpt-4o-mini"))

In [ ]:

@dataclass
class ContextInfo:
    auth_header: Optional[dict] = None
    data_type: Optional[str] = None
    date: str = ""  # str(date.today())
    restart: bool = False
    last_agent_name: Optional[str] = None

context=ContextInfo(
                auth_header={
                    "Authorization": f"Bearer XXX",
                    "Content-Type": "application/json",
                }
            )

async def handle_streaming_intermediate_steps(message: ChatMessageContent) -> None:
    for item in message.items or []:
        if isinstance(item, FunctionCallContent):
            print(f"Function Call:> {item.name} with arguments: {item.arguments}")

        elif isinstance(item, FunctionResultContent):
            #print(f"Function Result:> {item.result} for function: {item.name}")
            
            # Attempt to parse and update context
            try:
                # `item.result` is a list of TextContent objects, take the first and parse its `text`
                raw_text = item.result[0].text  # Assuming it's the first item
                result_dict = json.loads(raw_text)

                # Update context
                if "last_agent_name" in result_dict:
                    context.last_agent_name = result_dict["last_agent_name"]
                    print(f"> Updated context.last_agent_name to: {context.last_agent_name}")
            except Exception as e:
                print(f"Error parsing FunctionResultContent: {e}")
        
        else:
            print(f"{message.role}: {message.content}")
 

In [45]:
async def main():
    # Initialize a chat agent with basic instructions
    async with (
        MCPSsePlugin(
            name="server1",
            url="http://localhost:8000/sse",
        ) as server1,
        MCPSsePlugin(
            name="hhai_mcp_server",
            url= f"{AZURE_MCP_HHAI_ENDPOINT}/sse",
            headers= {"x-api-key": AZURE_MCP_HHAI_API_KEY},
        ) as hhai_mcp_server,
    ):        
        triage_agent = ChatCompletionAgent(
            kernel = kernel,
            instructions="""
                You interact with the user to help them with queries or booking request 
                You have dedicated tools for this, including one to answer health related question using the hhai_mcp_server plugin 
                You should always use plugin to reply the user.
                If user say:
                 1. i want to book/cancel/etc a vaccine
                 2. Recommend me vaccinations/ what vaccines i should take
                 3. Asking about past records or vaccination
                 4. Anything to do with query to do with booking vaccination appointment always use server1 plugin.
                    """,
            name="triage",
            plugins=[server1,hhai_mcp_server],
            )
        thread: ChatHistoryAgentThread = None
        # Get a response to a user message
  
        user_input = input(
            "Hi! I am here to help with your vaccination needs. What would you like to do? "
        )
        print("User:", user_input)
        while True:
            #print("testtttt",context.last_agent_name)
            print(f"# User: {user_input}", end="\n\n")
            first_chunk = True
            async for response in triage_agent.invoke_stream(
                messages=user_input,
                thread=thread,
                on_intermediate_message = handle_streaming_intermediate_steps
                
            ):
                if first_chunk:
                    print(f"# {response.name}: ", end="", flush=True)
                    first_chunk = False
                
                print(response.content, end="", flush=True)
                #print(response)
                thread = response.thread
            print()
            user_input = input("Enter a message (or type 'exit' to quit): ")
            print("User:", user_input)
            if user_input.lower() == "exit":
                print("Thank you for using the vaccination assistant. Goodbye!")
                break

asyncio.run(main())

User: what side effect of influenza vaccination
# User: what side effect of influenza vaccination

# triage: The side effects of the influenza vaccination can include:

1. **Common Side Effects**:
   - Soreness, redness, or swelling at the injection site
   - Muscle aches
   - Low-grade fever
   - Headache
   - Tiredness

2. **Less Common Side Effects**:
   - Itchy skin rashes
   - Runny nose
   - Cough
   - Vomiting (more common in children)

3. **Severe Reactions** (rare):
   - Swelling of the face or difficulty breathing

Most side effects resolve within 2-3 days. If you have concerns, it's best to consult your healthcare professional.

Would you like to learn how to manage any side effects from the influenza vaccination?Function Call:> hhai_mcp_server-healthhub_ai_tool with arguments: {"user_query":"What are the side effects of the influenza vaccination?"}
Error parsing FunctionResultContent: Expecting value: line 1 column 1 (char 0)

User: i want to book an influenza vaccination
#